### basic rnn


###  RNN with log softmax
$\text{LogSoftmax}(x)_i = \log\left(\frac{e^{x_i}}{\sum_{j} e^{x_j}}\right) $  
$Linear = xA^{T} + b $   
$h_{t} = Linear( cat(x_{t}, h_{t-1})) = Linear(x_{t}) + Linear(h_{t-1}) $  
$output = LogSoftmax( Linear(h_{t}) ) $   


In [1]:
import torch
import torch.nn as nn

#! basic rnn
#  assume x_t, t = input_size, 
#  we would also like to have h_{t-1} as input 
class RNN(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #append the hiden input to the x input
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [2]:
#Load data
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('name_gender_data/*.txt'))

import unicodedata
def get_chinese_characters():
    chinese_characters = []

    for code_point in range(0x4e00, 0x9fff + 1):  # Range for Chinese characters in Unicode
        char = chr(code_point)
        if 'CJK UNIFIED IDEOGRAPH' in unicodedata.name(char, ''):
            chinese_characters.append(char)

    return chinese_characters

all_letters = get_chinese_characters()
n_letters = len(all_letters)
print(f"number of letter == ",n_letters)

# # Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicodeToAscii('严'))

['name_gender_data\\female.txt', 'name_gender_data\\male.txt']
number of letter ==  20989
严


In [3]:
#vertify data
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('name_gender_data/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print("the number of categories == ", n_categories)
print(category_lines['male'][:5])

the number of categories ==  2
['建国', '建华', '国华', '和平', '明']


In [5]:
#word to vector

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.index(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('明'))
print(lineToTensor('建国').size())
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
input = lineToTensor('建国')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([2, 1, 20989])
tensor([[-0.6760, -0.7106]], grad_fn=<LogSoftmaxBackward0>)


In [6]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('female', 0)


In [7]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = male / line = 伟
category = female / line = 丽
category = female / line = 娟
category = male / line = 鑫
category = male / line = 宇航
category = female / line = 品妤
category = male / line = 建军
category = male / line = 明
category = female / line = 红
category = male / line = 皓轩


In [8]:
criterion = nn.NLLLoss()
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [10]:
import time
import math

n_iters = 10000
print_every = 2000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

2000 20% (0m 9s) 0.6771 俊杰 / male ✓
4000 40% (0m 17s) 0.6684 品妍 / female ✓
6000 60% (0m 24s) 0.2242 梓宣 / female ✓
8000 80% (0m 31s) 0.0195 子轩 / male ✓
10000 100% (0m 37s) 0.0403 梓宣 / female ✓


In [11]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

def predict(input_line, n_predictions=2):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict(u'小傻瓜')
predict(u'社恐')
predict('牛逼')
predict('小可爱')


> 小傻瓜
(-0.48) female
(-0.97) male

> 社恐
(-0.53) female
(-0.89) male

> 牛逼
(-0.52) female
(-0.90) male

> 小可爱
(-0.46) female
(-1.00) male
